In [ ]:
import io, os
input=io.BytesIO(os.read(0,os.fstat(0).st_size)).readline

import collections
def bfs(C, F, l, src, snk) :
  V = len(C)
  Q = collections.deque()
  Q.append(src)
  l[src] = 0
  while Q :
    u = Q.popleft()
    for v in range(V) :
      if l[v] == -1 and C[u][v] - F[u][v] > 0 :
        l[v] = l[u] + 1
        Q.append(v)
  return l[snk] != -1

def dfs(C, F, W, l, u, snk, cur = 1 << 63) :
  if u == snk : return cur
  V = len(C)
  while W[u] < V :
    v = W[u]
    if l[v] == l[u] + 1 and C[u][v] > F[u][v] :
      A = min(cur, C[u][v] - F[u][v])
      tmp = dfs(C, F, W, l, v, snk, A)
      if tmp > 0 :
        F[u][v] += tmp
        F[v][u] -= tmp
        return tmp
    W[u] += 1
  return 0

def dinic(C, src, snk) :
  V = len(C)
  F = [[0] * V for _ in range(V)]
  ret = 0

  while True :
    l = [-1] * V
    W = [0] * V
    if not bfs(C, F, l, src, snk) : break
    while True:
      tmp = dfs(C, F, W, l, src, snk)
      if tmp == 0 : break
      ret += tmp
  return ret, F

def dec(G, C, F) :
  Q = collections.deque([0])
  vis = [False] * len(C)
  vis[0] = True
  while Q :
    u = Q.popleft()
    for v in range(len(C)) :
      if not vis[v] and C[u][v] > F[u][v] : #포화간선을 거치지 않고 닿을 수 있는 모든 정점 = 도달 가능한 그룹
        vis[v] = True
        Q.append(v)
  return vis

def sol():
  N = int(input())
  X = [*map(int, input().split())] #사람 진영
  G = [[*map(int, input().split())] for _ in range(N)]
  
  V = N + 2
  src = 0
  sink = V - 1
  C = [[0] * V for _ in range(V)]
  for i, v in enumerate(X) :
    if v == 1 : C[src][i + 1] = 1 << 30
    if v == 2 : C[i + 1][sink] = 1 << 30

    for j, w in enumerate(G[i]) : #가중치 적용
      if i == j: continue
      C[i + 1][j + 1] = w

  ans, F = dinic(C, src, sink)
  print(ans)
  vis = dec(G, C, F)
  ans1 = []
  ans2 = []
  for i, v in enumerate(vis[1:-1], 1) :
    if v : ans1.append(i)
    else : ans2.append(i)
  print(*ans1) 
  print(*ans2) 
  
sol()

### 풀이
- 이분 그래프를 구성하는 법: `A`진영 사람들을 source, `B`진영 사람들을 sink와 연결한다. 그리고 가중치는 주워진 인접행렬을 그대로 사용한다.
- 그룹에 속한 사람들을 출력하는 방법: 도달 가능한 정점들의 집합이 `A`, 불가능한 정점들의 집합이 `B`이다. 
  - sink부터 bfs로 돌면서 포화간선을 거치지 않고 닿을 수 있는 모든 정점들을 기록하면 `A`를 구성하게 된다. 그 외가 `B`가 된다.

In [ ]:
import io, os, sys, collections
input=io.BytesIO(os.read(0,os.fstat(0).st_size)).readline

INF = float("inf")
class Dinic:
  def __init__(self, n):
    self.lvl = [0] * n
    self.ptr = [0] * n
    self.q = [0] * n
    self.adj = [[] for _ in range(n)]

  def add_edge(self, a, b, c, rcap=0):
    self.adj[a].append([b, len(self.adj[b]), c, 0])
    self.adj[b].append([a, len(self.adj[a]) - 1, rcap, 0])

  def dfs(self, v, t, f):
    if v == t or not f:
      return f

    for i in range(self.ptr[v], len(self.adj[v])):
      e = self.adj[v][i]
      if self.lvl[e[0]] == self.lvl[v] + 1:
        p = self.dfs(e[0], t, min(f, e[2] - e[3]))
        if p:
          self.adj[v][i][3] += p
          self.adj[e[0]][e[1]][3] -= p
          return p
      self.ptr[v] += 1

    return 0

  def calc(self, s, t):
    flow, self.q[0] = 0, s
    for l in range(20):
      while True:
        self.lvl, self.ptr = [0] * len(self.q), [0] * len(self.q)
        qi, qe, self.lvl[s] = 0, 1, 1
        while qi < qe and not self.lvl[t]:
          v = self.q[qi]
          qi += 1
          for e in self.adj[v]:
            if not self.lvl[e[0]] and (e[2] - e[3]) >> (19 - l):
              self.q[qe] = e[0]
              qe += 1
              self.lvl[e[0]] = self.lvl[v] + 1

        p = self.dfs(s, t, INF)
        while p:
          flow += p
          p = self.dfs(s, t, INF)

        if not self.lvl[t]:
          break

    return flow

def sol():
  N = int(input())
  X = [*map(int, input().split())] #사람 진영
  G = [[*map(int, input().split())] for _ in range(N)]
  
  V = N + 2
  src = 0
  sink = V - 1
  C = Dinic(V)
  for i, v in enumerate(X) :
    if v == 1 : C.add_edge(src, i + 1, 501000) #A 그룹에 반드시 속해야 하는 사람
    if v == 2 : C.add_edge(i + 1, sink, 501000)

    for j, w in enumerate(G[i]) : #가중치 적용
      if i == j: continue
      C.add_edge(i + 1, j + 1, w)

  ans = C.calc(src, sink)
  sys.stdout.write(str(ans) + '\n')

  Q = collections.deque([src])
  vis = [False] * V
  vis[0] = True
  while Q :
    u = Q.popleft()
    for v, _, c, f in C.adj[u] :
      if not vis[v] and c > f : #포화간선을 거치지 않고 닿을 수 있는 모든 정점 = 도달 가능한 그룹 = A
        vis[v] = True
        Q.append(v)
  
  ans1 = []
  ans2 = []
  for i, v in enumerate(vis[1:-1], 1) :
    if v : ans1.append(i)
    else : ans2.append(i)
  sys.stdout.write(' '.join(map(str, ans1)) + '\n')
  sys.stdout.write(' '.join(map(str, ans2)) + '\n')
  
sol()

- 개느리다. TLE. 템플릿이 느린건지 해 출력이 느린건지.